In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Install an End-To-End Closed Domain Question Answering System
!pip install cdqa

     |████████████████████████████████| 45 kB 1.7 MB/s 
     |████████████████████████████████| 94 kB 2.5 MB/s 
     |████████████████████████████████| 278 kB 41.4 MB/s 
     |████████████████████████████████| 10.4 MB 33.0 MB/s 
     |████████████████████████████████| 311 kB 47.3 MB/s 
     |████████████████████████████████| 6.7 MB 53.3 MB/s 
     |████████████████████████████████| 748.9 MB 703 bytes/s 
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 131 kB 40.3 MB/s 
     |████████████████████████████████| 1.2 MB 34.4 MB/s 
     |████████████████████████████████| 8.2 MB 38.4 MB/s 
     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 138 kB 49.1 MB/s 
     |████████████████████████████████| 127 kB 49.1 MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-py3-none-any.whl size=47651 sha

In [1]:
from ast import literal_eval
from cdqa.pipeline import QAPipeline
from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
from nltk.parse import load_parser
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import time
import textwrap
import re
import attr
import abc
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import HTML
from os import listdir
from os.path import isfile, join


In [3]:
df = pd.read_csv("/content/drive/MyDrive/CSC494_NLP/Bibliometrix-melioidosis-2021-11-23.csv")
df.head(-10)

,AU,AF,DE,AID,OT,PHST,DT,AB,C1,OI,BTI,CI,CIN,CN,COIS,CON,CP,CRDT,CRF,CRI,CTDT,DCOM,DEP,PY,ED,EDAT,EFR,EIN,EN,FED,FIR,FPS,GN,GR,IS,IR,IRAD,SN,ISBN,JID,...,MHDA,MID,OAB,OABL,OID,OTO,OWN,PB,PP,PL,PMC,PMCR,PMID,PS,PST,RF,RIN,RN,ROF,SB,SI,SO2,STAT,J9,TI,TT,UIN,UOF,VL,DI,DB,ID,RP,TC,CR,AU_UN,AU1_UN,AU_UN_NR,SR_FULL,SR
0,CHAKRAVORTY A;HEATH CH,"CHAKRAVORTY, ARINDAM;HEATH, CHRISTOPHER",ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,10.31128/AJGP-04-18-4558 [DOI],NaN,2019/05/28 06:00 [ENTREZ];2019/05/28 06:00 [PU...,JOURNAL ARTICLE;REVIEW,BACKGROUND: ENDEMIC TO SOILS OF NORTHERN AUSTR...,"MBBS, FRACP, DEPARTMENT OF MICROBIOLOGY AND IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/05/28 06:00,NaN,NaN,NaN,20200616.0,NaN,2019,NaN,2019/05/28 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,2208-7958 (ELECTRONIC),NaN,101718099,...,2020/06/17 06:00,NaN,NaN,NaN,NaN,NaN,NLM,NaN,327-332,AUSTRALIA,NaN,NaN,31129946,NaN,PPUBLISH,NaN,NaN,0 (ANTI-BACTERIAL AGENTS),NaN,IM,NaN,AUST J GEN PRACT. 2019 MAY;48(5):327-332. DOI:...,MEDLINE,AUST J GEN PRACT,MELIOIDOSIS: AN UPDATED REVIEW.,NaN,NaN,NaN,48,10.31128/AJGP-04-18-4558,PUBMED,ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,NaN,0,none,ROYAL PERTH HOSPITAL;CHAKRAVORTY@HEALTH.;ROYAL...,ROYAL PERTH HOSPITAL;NOTREPORTED;CHAKRAVORTY@H...,NaN,"CHAKRAVORTY A, 2019, AUST J GEN PRACT","CHAKRAVORTY A, 2019, AUST J GEN PRACT"
1,VIRK HS;MUKHOPADHYAY C;WIERSINGA WJ,"VIRK, HARJEET SINGH;MUKHOPADHYAY, CHIRANJAY;WI...",ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;BURKHOLD...,10.1055/S-0040-1710570 [DOI],NaN,2020/07/07 06:00 [ENTREZ];2020/07/07 06:00 [PU...,JOURNAL ARTICLE;REVIEW,"MELIOIDOSIS, CAUSED BY THE FACULTATIVE INTRACE...",CENTER FOR EXPERIMENTAL AND MOLECULAR MEDICINE...,NaN,NaN,"THIEME MEDICAL PUBLISHERS 333 SEVENTH AVENUE, ...",NaN,NaN,NONE DECLARED.,NaN,NaN,2020/07/07 06:00,NaN,NaN,NaN,20210712.0,20200706.0,2020,NaN,2020/07/07 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NETHERLANDS ORGANIZATION FOR SCIENTIFIC RESEAR...,4,NaN,NaN,1098-9048 (ELECTRONIC) 1069-3424 (LINKING),NaN,9431858,...,2021/07/13 06:00,NaN,NaN,NaN,NaN,NaN,NLM,NaN,496-508,UNITED STATES,NaN,NaN,32629488,NaN,PPUBLISH,NaN,NaN,0 (ANTI-BACTERIAL AGENTS) 0 (VACCINES),NaN,IM,NaN,SEMIN RESPIR CRIT CARE MED. 2020 AUG;41(4):496...,MEDLINE,SEMIN RESPIR CRIT CARE MED,MELIOIDOSIS: A NEGLECTED CAUSE OF COMMUNITY-AC...,NaN,NaN,NaN,41,10.1055/S-0040-1710570,PUBMED,ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;BURKHOLD...,NaN,0,none,AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...,AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...,NaN,"VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED","VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED"
2,PATTAMAPASPONG N;MUTTARAK M,"PATTAMAPASPONG, NUTTAYA;MUTTARAK, MALAI","DIAGNOSIS, DIFFERENTIAL;DIAGNOSTIC IMAGING;HUM...",10.1055/S-0031-1293494 [DOI],NaN,2011/11/15 06:00 [ENTREZ];2011/11/15 06:00 [PU...,JOURNAL ARTICLE;REVIEW,MELIOIDOSIS IS AN INFECTIOUS DISEASE CAUSED BY...,"DEPARTMENT OF RADIOLOGY, FACULTY OF MEDICINE, ...",NaN,NaN,THIEME MEDICAL PUBLISHERS.,NaN,NaN,NaN,NaN,NaN,2011/11/15 06:00,NaN,NaN,NaN,20120403.0,20111111.0,2011,NaN,2011/11/15 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,1098-898X (ELECTRONIC) 1089-7860 (LINKING),NaN,9717520,...,2012/04/04 06:00,NaN,NaN,NaN,NaN,NaN,NLM,NaN,480-8,UNITED STATES,NaN,NaN,22081283,NaN,PPUBLISH,NaN,NaN,NaN,NaN,IM,NaN,SEMIN MUSCULOSKELET RADIOL. 2011 NOV;15(5):480...,MEDLINE,SEMIN MUSCULOSKELET RADIOL,MUSCULOSKELETAL MELIOIDOSIS.,NaN,NaN,NaN,15,10.1055/S-0031-1293494,PUBMED,"DIAGNOSIS, DIFFERENTIAL;DIAGNOSTIC IMAGING;HUM...",NaN,0,none,CHIANG MAI UNIVERSITY,CHIANG MAI UNIVERSITY;NOTREPORTED;NOTREPORTED,NaN,"PATTAMAPASPONG N, 2011, SEMIN MUSCULOSKELET RA...","PATTAMAPASPONG N, 2011, SEMIN MUSCULOSKELET RA..."
3,LE TOHIC S;MONTANA M;KOCH L;CURTI C;VANELLE P,"LE TOHIC, SARAH;MONTANA, MARC;KOCH, LIONEL;CUR...",BURKHOLDERIA PSEUDOMALLEI;COMMUNICABLE DISEASE...,10.1007/S10096-019-03548-5 [PII];10.1007/S1009...,CASE REPORTS;MELIOIDOSIS;PNEUMONIA;TRAVEL MEDI...,2019/02/08 00:00 [RECEIVED];2019/03/25 00:00 [...,JOURN

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3202 entries, 0 to 3201
Data columns (total 84 columns):
AU          3187 non-null object
AF          3187 non-null object
DE          2841 non-null object
AID         2635 non-null object
OT          790 non-null object
PHST        3197 non-null object
DT          3202 non-null object
AB          2607 non-null object
C1          2561 non-null object
OI          307 non-null object
BTI         5 non-null object
CI          481 non-null object
CIN         78 non-null object
CN          16 non-null object
COIS        520 non-null object
CON         69 non-null object
CP          2 non-null object
CRDT        3202 non-null object
CRF         1 non-null object
CRI         1 non-null object
CTDT        4 non-null float64
DCOM        2948 non-null float64
DEP         1432 non-null float64
PY          3202 non-null int64
ED          2 non-null object
EDAT        3202 non-null object
EFR         5 non-null object
EIN         39 non-null object


In [5]:
df.iloc[1]

AU                        VIRK HS;MUKHOPADHYAY C;WIERSINGA WJ
AF          VIRK, HARJEET SINGH;MUKHOPADHYAY, CHIRANJAY;WI...
DE          ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;BURKHOLD...
AID                              10.1055/S-0040-1710570 [DOI]
OT                                                        NaN
                                  ...                        
AU_UN       AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...
AU1_UN      AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...
AU_UN_NR                                                  NaN
SR_FULL             VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED
SR                  VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED
Name: 1, Length: 84, dtype: object

In [6]:
df.shape

(3202, 84)

In [7]:
df.AB.value_counts

<bound method IndexOpsMixin.value_counts of 0       BACKGROUND: ENDEMIC TO SOILS OF NORTHERN AUSTR...
1       MELIOIDOSIS, CAUSED BY THE FACULTATIVE INTRACE...
2       MELIOIDOSIS IS AN INFECTIOUS DISEASE CAUSED BY...
3       MELIOIDOSIS IS A TROPICAL BACTERIAL INFECTION,...
4       MELIOIDOSIS, WHICH IS INFECTION WITH THE GRAM-...
                              ...                        
3197                                                  NaN
3198                                                  NaN
3199                                                  NaN
3200                                                  NaN
3201                                                  NaN
Name: AB, Length: 3202, dtype: object>

In [8]:
df.AB.value_counts()

GLANDERS AND MELIOIDOSIS ARE TWO INFECTIOUS DISEASES THAT ARE CAUSED BY BURKHOLDERIA MALLEI AND BURKHOLDERIA PSEUDOMALLEI RESPECTIVELY. INFECTION MAY BE ACQUIRED THROUGH DIRECT SKIN CONTACT WITH CONTAMINATED SOIL OR WATER. INGESTION OF SUCH CONTAMINATED WATER OR DUST IS ANOTHER WAY OF CONTAMINATION. GLANDERS AND MELIOIDOSIS HAVE BOTH BEEN STUDIED FOR WEAPONISATION IN SEVERAL COUNTRIES IN THE PAST. THEY PRODUCE SIMILAR CLINICAL SYNDROMES. THE SYMPTOMS DEPEND UPON THE ROUTE OF INFECTION BUT ONE FORM OF THE DISEASE MAY PROGRESS TO ANOTHER, OR THE DISEASE MIGHT RUN A CHRONIC RELAPSING COURSE. FOUR CLINICAL FORMS ARE GENERALLY DESCRIBED: LOCALISED INFECTION, PULMONARY INFECTION, SEPTICAEMIA AND CHRONIC SUPPURATIVE INFECTIONS OF THE SKIN. ALL TREATMENT RECOMMENDATIONS SHOULD BE ADAPTED ACCORDING TO THE SUSCEPTIBILITY REPORTS FROM ANY ISOLATES OBTAINED. POST-EXPOSURE PROPHYLAXIS WITH TRIMETHOPRIM-SULFAMETHOXAZOLE IS RECOMMENDED IN CASE OF A BIOLOGICAL ATTACK. THERE IS NO VACCINE AVAILABLE FOR

In [9]:
print("Abstract example","\n")
print(df.AB.values[2])
print(df.AU.values[2])
print(df.AU_UN.values[2])
print(df.DE.values[2])
print(df.DT.values[2])

Abstract example 

MELIOIDOSIS IS AN INFECTIOUS DISEASE CAUSED BY BURKHOLDERIA PSEUDOMALLEI, MOSTLY AFFECTING PATIENTS IN SOUTHEAST ASIA AND NORTHERN AUSTRALIA. THE DISEASE HAS BEEN INCREASINGLY RECOGNIZED AROUND THE WORLD DUE TO THE INCREASED LEVELS OF TRAVEL AND POPULATION MOVEMENT. CLINICAL MANIFESTATIONS OF MELIOIDOSIS RANGE FROM FULMINANT SEPTICEMIC ILLNESS TO AN INDOLENT LOCAL INFECTION. THE DISEASE OFTEN INVOLVES MULTIPLE ORGANS, INCLUDING THE LUNG, SPLEEN, LIVER, AND OTHER VISCERAL ORGANS. MUSCULOSKELETAL INFECTION IS USUALLY SEEN AS A PART OF MULTIORGAN INVOLVEMENT, BUT LOCALIZED MUSCULOSKELETAL INVOLVEMENT MAY OCCUR. THE MOST COMMON MANIFESTATION OF MUSCULOSKELETAL MELIOIDOSIS IS SEPTIC ARTHRITIS, FOLLOWED BY OSTEOMYELITIS, PYOMYOSITIS, AND SOFT TISSUE ABSCESSES. THE CLINICAL AND RADIOLOGICAL MANIFESTATIONS OF MUSCULOSKELETAL MELIOIDOSIS ARE NONSPECIFIC, AND THE DIAGNOSIS NEEDS A HIGH LEVEL OF SUSPICION. ASSOCIATED INFECTION OF LUNGS AND VISCERAL ORGANS IS SUGGESTIVE OF MELIO

In [10]:
df.describe(include='all')

,AU,AF,DE,AID,OT,PHST,DT,AB,C1,OI,BTI,CI,CIN,CN,COIS,CON,CP,CRDT,CRF,CRI,CTDT,DCOM,DEP,PY,ED,EDAT,EFR,EIN,EN,FED,FIR,FPS,GN,GR,IS,IR,IRAD,SN,ISBN,JID,...,MHDA,MID,OAB,OABL,OID,OTO,OWN,PB,PP,PL,PMC,PMCR,PMID,PS,PST,RF,RIN,RN,ROF,SB,SI,SO2,STAT,J9,TI,TT,UIN,UOF,VL,DI,DB,ID,RP,TC,CR,AU_UN,AU1_UN,AU_UN_NR,SR_FULL,SR
count,3187,3187,2841,2635,790,3197,3202,2607,2561,307,5,481,78,16,520,69,2,3202,1,1,4.000000e+00,2.948000e+03,1.432000e+03,3202.000000,2,3202,5,39,1,2,5,4,1,591,2819,5,1,3180,2,3197,...,3197,61,7,7,61,790,3197,5,3019,1803,1399,13,3.202000e+03,4,3197,122.000000,1,1868,1,2882,72,3197,3202,3197,3202,173,2,2,3090,1698,3202,2841,0.0,3202.0,3202,2536,3202,0.0,3202,3202
unique,3092,3105,2814,2635,750,2807,143,2606,2371,270,3,339,77,12,159,64,2,2367,1,1,NaN,NaN,NaN,NaN,2,2301,5,39,1,2,5,4,1,351,122,5,1,917,2,745,...,2146,61,4,3,61,4,1,3,2822,51,1399,12,NaN,3,3,NaN,1,1153,1,15,72,3197,5,745,3143,170,2,2,296,1698,1,2814,NaN,NaN,1,1638,1801,NaN,3150,3202
top,ASHDOWN LR,"ASHDOWN, L R",HUMANS;MELIOIDOSIS,10.1111/J.1445-5994.1976.TB03312.X [DOI],*MELIOIDOSIS,2009/01/06 09:00 [ENTREZ];2009/01/16 09:00 [PU...,JOURNAL ARTICLE,GLANDERS AND MELIOIDOSIS ARE TWO INFECTIOUS DI...,"FACULTY OF TROPICAL MEDICINE, MAHIDOL UNIVERSI...",ORCID: 0000-0001-9189-7244,STATPEARLS,THE AMERICAN SOCIETY OF TROPICAL MEDICINE AND...,AM J TROP MED HYG. 2014 FEB;90(2):386. PMID: 2...,CENTERS FOR DISEASE CONTROL AND PREVENTION (CDC),THE AUTHORS HAVE DECLARED THAT NO COMPETING IN...,LANCET INFECT DIS. 2019 AUG;19(8):892-902. PMI...,CHAPTER 27,2009/01/06 09:00,BR J CLIN PHARMACOL. 2000 MAY;49(5):445-52. PM...,BR J CLIN PHARMACOL. 2000 AUG;50(2):184-91. PM...,NaN,NaN,NaN,NaN,BARON S,2009/01/16 09:00,PLOS NEGL TROP DIS. 2015 MAR;9(3):E0003586. PM...,LANCET 1995 OCT 28;346(8983):1172,4TH,"DONDORP, ARJEN M DNSER, MARTIN W SCHULTZ, MARC...","ROZEMEIJER, WOUTER RIJKEBOER, ANNEMIEKE BIRNIE...","MORRELL, JAMES",PIP: TJ: JAPANESE JOURNAL OF MEDICAL SCIENCE &...,WELLCOME TRUST/UNITED KINGDOM,1,ADAMS A BAKER K BARANSI K CASSITY TR CURTIN J ...,"DEPARTMENT OF MEDICAL MICROBIOLOGY, NORTHWEST ...",1935-2735 (ELECTRONIC) 1935-2727 (PRINT) 1935-...,0-9631172-1-1,0370507,...,2009/11/11 06:00,NIHMS398142,PUBLISHER: ABSTRACT AVAILABLE FROM THE PUBLISHER.,ENG,CLML: 5732:26807,NOTNLM,NLM,STATPEARLS PUBLISHING,335-7,ENGLAND,PMC5988316,2022/04/01,NaN,DOYLE AC,PPUBLISH,NaN,PLOS NEGL TROP DIS. 2020 JUL 1;14(7):E0008441....,0 (ANTI-BACTERIAL AGENTS),PLOS NEGL TROP DIS. 2017 MAY 11;11(5):E0004738...,IM,GENBANK/U29876 GENBANK/U29877 GENBANK/U29878 G...,J INFECT DIS. 1970 FEB;121(2):95-102. DOI: 10....,MEDLINE,AM J TROP MED HYG,MELIOIDOSIS.,MELIOIDOSE NUMA PEQUENA COMUNIDADE RURAL DO CEAR.,NPJ VACCINES. 2021 MAR 30;6(1):47. PMID: 33785745,BIORXIV. 2020 NOV 18;:. PMID: 33236013,10,10.1586/14760584.2016.1170598,PUBMED,HUMANS;MELIOIDOSIS,NaN,NaN,none,MAHIDOL UNIVERSITY,NOTREPORTED,NaN,"REECHAIPICHITKUL W, 2004, SOUTHEAST ASIAN J TR...","LI Q, 2015, AUTOPHAGY"
freq,12,12,14,1,20,26,807,2,26,6,3,19,2,3,135,3,1,26,1,1,NaN,NaN,NaN,NaN,1,27,1,1,1,1,1,1,1,142,442,1,1,114,1,144,...,29,1,4,3,1,707,3197,3,6,470,1,2,NaN,2,2554,NaN,1,240,1,2439,1,1,2841,144,25,2,1,1,90,1,3202,14,NaN,NaN,3202,154,657,NaN,3,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.020562e+07,2.006587e+07,2.014915e+07,2005.740162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.987233e+07,NaN,NaN,53.196721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.027345e+04,1.232976e+05,4.396059e+04,15.015142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.431190e+06,NaN,NaN,44.574692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [11]:
print("Abstracts:","\n")
print(df.AB.values[:])

Abstracts: 

['BACKGROUND: ENDEMIC TO SOILS OF NORTHERN AUSTRALIA, BURKHOLDERIA PSEUDOMALLEI IS THE GRAM-NEGATIVE PATHOGEN RESPONSIBLE FOR MELIOIDOSIS, WHICH CAUSES A CLINICAL SPECTRUM RANGING FROM PNEUMONIA AND/OR CUTANEOUS INFECTION TO DISSEMINATED DISEASE WITH FULMINANT SEPTICAEMIA. INCIDENT CASES PEAK AFTER MONSOONAL RAINS, PARTICULARLY IN INDIVIDUALS WITH IMMUNE DYSFUNCTION. EARLY DIAGNOSIS OF MELIOIDOSIS IS CHALLENGING FOR CLINICIANS, GIVEN ITS ABILITY TO MIMIC MANY OTHER DISEASES AND HIGH CLINICAL (ASSOCIATED MORTALITY) IMPACT. OBJECTIVES: THE AIM OF THIS PAPER IS TO PROVIDE GENERAL PRACTITIONERS WITH AN OVERVIEW OF MELIOIDOSIS, COVERING EPIDEMIOLOGY, RISK FACTORS FOR INFECTION, CLINICAL DISEASE SPECTRUM, DIAGNOSTIC TECHNIQUES AND AN APPROACH TO MANAGEMENT, INCLUDING PUBLIC HEALTH ASPECTS. DISCUSSION: PRIMARY CARE PHYSICIANS PLAY A KEY ROLE IN EARLY DISEASE RECOGNITION, INITIAL PATIENT STABILISATION, REQUEST OF APPROPRIATE CLINICAL SAMPLES (PARTICULARLY BLOOD CULTURES) AND PROMP

In [12]:
## Remove punctuation from each text:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['TI'] = df['TI'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['TI'] = df['TI'].apply(lambda x: lower_case(x))

df['AB'] = df['AB'].apply(str)
df['AB'] = df['AB'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['AB'] = df['AB'].apply(lambda x: lower_case(x))

In [13]:
df['AB'].head()

0    background endemic to soils of northern austra...
1    melioidosis caused by the facultative intracel...
2    melioidosis is an infectious disease caused by...
3    melioidosis is a tropical bacterial infection ...
4    melioidosis which is infection with the gramne...
Name: AB, dtype: object

In [14]:
text = df[["TI"]]
text.head()

,TI
0,melioidosis an updated review
1,melioidosis a neglected cause of communityacqu...
2,musculoskeletal melioidosis
3,a review of melioidosis cases imported into eu...
4,melioidosis


In [15]:
## IPython Utilities
from IPython.display import HTML

import notebook as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output, VBox

from IPython.html import widgets
from IPython.display import display, Image, HTML, Markdown, clear_output

def printmd(string):
    display(Markdown(string))

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [16]:
download_model(model='bert-squad_1.1', dir='./models')

In [17]:
df["paragraphs"] = [x.split('\n') for x in df["AB"]]

In [18]:
df = filter_paragraphs(df)
df.head()

,AU,AF,DE,AID,OT,PHST,DT,AB,C1,OI,BTI,CI,CIN,CN,COIS,CON,CP,CRDT,CRF,CRI,CTDT,DCOM,DEP,PY,ED,EDAT,EFR,EIN,EN,FED,FIR,FPS,GN,GR,IS,IR,IRAD,SN,ISBN,JID,...,MID,OAB,OABL,OID,OTO,OWN,PB,PP,PL,PMC,PMCR,PMID,PS,PST,RF,RIN,RN,ROF,SB,SI,SO2,STAT,J9,TI,TT,UIN,UOF,VL,DI,DB,ID,RP,TC,CR,AU_UN,AU1_UN,AU_UN_NR,SR_FULL,SR,paragraphs
0,CHAKRAVORTY A;HEATH CH,"CHAKRAVORTY, ARINDAM;HEATH, CHRISTOPHER",ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,10.31128/AJGP-04-18-4558 [DOI],NaN,2019/05/28 06:00 [ENTREZ];2019/05/28 06:00 [PU...,JOURNAL ARTICLE;REVIEW,background endemic to soils of northern austra...,"MBBS, FRACP, DEPARTMENT OF MICROBIOLOGY AND IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/05/28 06:00,NaN,NaN,NaN,20200616.0,NaN,2019,NaN,2019/05/28 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,2208-7958 (ELECTRONIC),NaN,101718099,...,NaN,NaN,NaN,NaN,NaN,NLM,NaN,327-332,AUSTRALIA,NaN,NaN,31129946,NaN,PPUBLISH,NaN,NaN,0 (ANTI-BACTERIAL AGENTS),NaN,IM,NaN,AUST J GEN PRACT. 2019 MAY;48(5):327-332. DOI:...,MEDLINE,AUST J GEN PRACT,melioidosis an updated review,NaN,NaN,NaN,48,10.31128/AJGP-04-18-4558,PUBMED,ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,NaN,0,none,ROYAL PERTH HOSPITAL;CHAKRAVORTY@HEALTH.;ROYAL...,ROYAL PERTH HOSPITAL;NOTREPORTED;CHAKRAVORTY@H...,NaN,"CHAKRAVORTY A, 2019, AUST J GEN PRACT","CHAKRAVORTY A, 2019, AUST J GEN PRACT",[background endemic to soils of northern austr...
1,VIRK HS;MUKHOPADHYAY C;WIERSINGA WJ,"VIRK, HARJEET SINGH;MUKHOPADHYAY, CHIRANJAY;WI...",ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;BURKHOLD...,10.1055/S-0040-1710570 [DOI],NaN,2020/07/07 06:00 [ENTREZ];2020/07/07 06:00 [PU...,JOURNAL ARTICLE;REVIEW,melioidosis caused by the facultative intracel...,CENTER FOR EXPERIMENTAL AND MOLECULAR MEDICINE...,NaN,NaN,"THIEME MEDICAL PUBLISHERS 333 SEVENTH AVENUE, ...",NaN,NaN,NONE DECLARED.,NaN,NaN,2020/07/07 06:00,NaN,NaN,NaN,20210712.0,20200706.0,2020,NaN,2020/07/07 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NETHERLANDS ORGANIZATION FOR SCIENTIFIC RESEAR...,4,NaN,NaN,1098-9048 (ELECTRONIC) 1069-3424 (LINKING),NaN,9431858,...,NaN,NaN,NaN,NaN,NaN,NLM,NaN,496-508,UNITED STATES,NaN,NaN,32629488,NaN,PPUBLISH,NaN,NaN,0 (ANTI-BACTERIAL AGENTS) 0 (VACCINES),NaN,IM,NaN,SEMIN RESPIR CRIT CARE MED. 2020 AUG;41(4):496...,MEDLINE,SEMIN RESPIR CRIT CARE MED,melioidosis a neglected cause of communityacqu...,NaN,NaN,NaN,41,10.1055/S-0040-1710570,PUBMED,ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;BURKHOLD...,NaN,0,none,AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...,AMSTERDAM UNIVERSITY MEDICAL CENTERS;KASTURBA ...,NaN,"VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED","VIRK HS, 2020, SEMIN RESPIR CRIT CARE MED",[melioidosis caused by the facultative intrace...
2,PATTAMAPASPONG N;MUTTARAK M,"PATTAMAPASPONG, NUTTAYA;MUTTARAK, MALAI","DIAGNOSIS, DIFFERENTIAL;DIAGNOSTIC IMAGING;HUM...",10.1055/S-0031-1293494 [DOI],NaN,2011/11/15 06:00 [ENTREZ];2011/11/15 06:00 [PU...,JOURNAL ARTICLE;REVIEW,melioidosis is an infectious disease caused by...,"DEPARTMENT OF RADIOLOGY, FACULTY OF MEDICINE, ...",NaN,NaN,THIEME MEDICAL PUBLISHERS.,NaN,NaN,NaN,NaN,NaN,2011/11/15 06:00,NaN,NaN,NaN,20120403.0,20111111.0,2011,NaN,2011/11/15 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,1098-898X (ELECTRONIC) 1089-7860 (LINKING),NaN,9717520,...,NaN,NaN,NaN,NaN,NaN,NLM,NaN,480-8,UNITED STATES,NaN,NaN,22081283,NaN,PPUBLISH,NaN,NaN,NaN,NaN,IM,NaN,SEMIN MUSCULOSKELET RADIOL. 2011 NOV;15(5):480...,MEDLINE,SEMIN MUSCULOSKELET RADIOL,musculoskeletal melioidosis,NaN,NaN,NaN,15,10.1055/S-0031-1293494,PUBMED,"DIAGNOSIS, DIFFERENTIAL;DIAGNOSTIC IMAGING;HUM...",NaN,0,none,CHIANG MAI UNIVERSITY,CHIANG MAI UNIVERSITY;NOTREPORTED;NOTREPORTED,NaN,"PATTAMAPASPONG N, 2011, SEMIN MUSCULOSKELET RA...","PATTAMAPASPONG N, 2011, SEMIN MUSCULOSKELET RA...",[melioidosis is an infectious disease caused b...
3,LE TOHIC S;MONTANA M;KOCH L;CURTI C;VANELLE P,"LE TOHIC, SARAH;MONTANA, MARC;KOCH, LIONEL;CUR...",BURKHOLDERIA PSEUDOMALLEI;COMMUNICABLE DISEASE...,10.1007/S10096-019-03548-5 [PII];10.1007/S1009...,CASE 

In [19]:
df = df.rename(columns={"TI": "title", "AB": "abstract"})
df.head(1)

,AU,AF,DE,AID,OT,PHST,DT,abstract,C1,OI,BTI,CI,CIN,CN,COIS,CON,CP,CRDT,CRF,CRI,CTDT,DCOM,DEP,PY,ED,EDAT,EFR,EIN,EN,FED,FIR,FPS,GN,GR,IS,IR,IRAD,SN,ISBN,JID,...,MID,OAB,OABL,OID,OTO,OWN,PB,PP,PL,PMC,PMCR,PMID,PS,PST,RF,RIN,RN,ROF,SB,SI,SO2,STAT,J9,title,TT,UIN,UOF,VL,DI,DB,ID,RP,TC,CR,AU_UN,AU1_UN,AU_UN_NR,SR_FULL,SR,paragraphs
0,CHAKRAVORTY A;HEATH CH,"CHAKRAVORTY, ARINDAM;HEATH, CHRISTOPHER",ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,10.31128/AJGP-04-18-4558 [DOI],NaN,2019/05/28 06:00 [ENTREZ];2019/05/28 06:00 [PU...,JOURNAL ARTICLE;REVIEW,background endemic to soils of northern austra...,"MBBS, FRACP, DEPARTMENT OF MICROBIOLOGY AND IN...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/05/28 06:00,NaN,NaN,NaN,20200616.0,NaN,2019,NaN,2019/05/28 06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,2208-7958 (ELECTRONIC),NaN,101718099,...,NaN,NaN,NaN,NaN,NaN,NLM,NaN,327-332,AUSTRALIA,NaN,NaN,31129946,NaN,PPUBLISH,NaN,NaN,0 (ANTI-BACTERIAL AGENTS),NaN,IM,NaN,AUST J GEN PRACT. 2019 MAY;48(5):327-332. DOI:...,MEDLINE,AUST J GEN PRACT,melioidosis an updated review,NaN,NaN,NaN,48,10.31128/AJGP-04-18-4558,PUBMED,ADULT;ANTI-BACTERIAL AGENTS/THERAPEUTIC USE;AU...,NaN,0,none,ROYAL PERTH HOSPITAL;CHAKRAVORTY@HEALTH.;ROYAL...,ROYAL PERTH HOSPITAL;NOTREPORTED;CHAKRAVORTY@H...,NaN,"CHAKRAVORTY A, 2019, AUST J GEN PRACT","CHAKRAVORTY A, 2019, AUST J GEN PRACT",[background endemic to soils of northern austr...


In [20]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 1671541.08B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [21]:
def get_cdqa_prediction(x):
    prediction = cdqa_pipeline.predict(x)
    question = '<h3>Question:</h3>'+x
    answer = '<h3>Answer:</h3>'+prediction[0]
    title = '<h3>Title:</h3>'+prediction[1]    
    paragraph = '<h3>Paragraph:</h3>'+prediction[2]    
    
    display(HTML(question))
    display(HTML(answer))
    display(HTML(title))
    display(HTML(paragraph))

In [26]:
text = widgets.Text(
    value='What do we know about diagnostics?',
    placeholder='Paste ticket description here!',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='50%', height='50px')
)

out = widgets.Output()

def callback(_):
    with out:
        clear_output()
        # what happens when we press the button
        printmd("**<font color=orange> ------------------------------------------------------------------------------------------------------------------------------- </font>**")        
        printmd(f"**<font color=blue>Melioidosis (Question & Answering System)</font>**")
        get_cdqa_prediction(text.value)
        printmd("**<font color=orange> ------------------------------------------------------------------------------------------------------------------------------- </font>**")        

text.on_submit(callback)
# displaying button and its output together
widgets.VBox([text, out])